In [3]:
import os
import ollama
# from gnews import GNews
from googlesearch import search
import requests
import os
from openai import OpenAI
from bs4 import BeautifulSoup
import re
import json

In [4]:
#VARIABLES
MODEL_NAME = "grok-beta"

In [5]:
def search_web(query):
    s = search(query, num_results=10,advanced=True)
    ret_str = ""
    for i in s:
        ret_str += i.description + "\n"
    return ret_str

In [6]:
functions = [
    {
        "name": "search_web",
        "description": "Search the web for new articles and information about the given topic",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The topic you want to search for",
                    "example_value": "SpaceX Current CEO",
                },
            },
            "required": ["query"],
            "optional": [],
        },
    },
    # {
    #     "name": "click",
    #     "description": "Click any button on a website and returns the new HTML",
    #     "parameters": {
    #         "type": "object",
    #         "properties": {
    #             "html": {
    #                 "type": "string",
    #                 "description": "A HTML",
    #             },
    #             "button": {
    #                 "type": "string",
    #                 "description": "A text description of a button on the html page",
    #             },
    #         },
    #         "required": ["html", "button"],
    #         "optional": [],
    #     },
    # },
]


In [13]:
messages=[
        {"role": "system", "content": '''You are a helpful assistant. Answer questions honestly and truthfully'''},
        {"role": "user", "content": '''Did the starship's 6th test flight already happen?'''}
    ]

In [14]:
client = OpenAI(
    api_key="xai-UiOih20Ae5VKxD7MGb2llRT5N4VMev11oTFnMS7rjsOuxtqOz7dyvS2oNBbMdrmoEM00vWYuZPQ5zrqX",
    base_url="https://api.x.ai/v1",
)

tools = [{'type': "function", "function": f} for f in functions]

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages,
    # tools=tools
)

response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="As of my last update in **2023**, Starship's 6th test flight has not occurred. Here are some key points regarding Starship's test flight history:\n\n- **First Test Flight**: Took place on April 20, 2023, but ended in an explosion shortly after lift-off due to a failure in stage separation.\n- **Second Test Flight**: Was set to occur later in 2023, but as of my last update, there were delays and no confirmation of the flight happening.\n\nFor the most current information:\n\n- **Check SpaceX's Official Channels**: SpaceX typically provides updates through their official website, X posts, and press releases.\n- **News Outlets**: Follow aerospace news outlets like SpaceNews, Ars Technica, or NASA Spaceflight for the latest on Starship developments.\n\nPlease verify with these sources for any updates or announcements regarding the 6th test flight.", refusal=None, role='assistant', audio=None, functi

In [25]:
tool_call = response.choices[0].message.tool_calls[0]

query = json.loads(tool_call.function.arguments)["query"]

html = search_web(query)
print("html is", html)

html is Fearless (Taylor's Version): The I Remember What You Said Last Night Chapter. Released: May 24, 2021; Label: Republic. Fearless (Taylor's Version): The From ...
8 Jan 2024 — 15. The Tortured Poets Department (April 2024) - Taylor Swift's New Album. During her 2024 Grammys acceptance speech for Best Pop Vocal Album ...
28 Oct 2024 — Swift's latest album, The Tortured Poets Department, was released on 19th April and became the first album in Spotify history to reach more ...
19 Apr 2024 — Speak Now (Taylor's Version) is Taylor's most recent release and is the third re-recording of her original material. It was released on 7th July ...
29 Oct 2024 — Every Taylor Swift Album In Original Release Order ; Evermore. 2020 ; Fearless (Taylor's Version). 2021 ; Red (Taylor's Version). 2021 ; Midnights.
2 May 2024 — By releasing two albums in one, Taylor Swift sacrificed quality for quantity in "The Tortured Poets Department."
Taylor Swift The Tortured Poets Department Available Now Stream

In [26]:
function_call_result_message = {
    "role": "tool",
    "content": html,
    "tool_call_id": response.choices[0].message.tool_calls[0].id
}

messages = [
     {"role": "system", "content": '''You are a helpful assistant. Answer questions honestly and truthfully. Use functions available to you to find information (if needed).'''},
    {"role": "user", "content": '''What is the latest taylow swift album?'''},
    response.choices[0].message,    
    function_call_result_message
]

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages,
    tools=tools,
)


In [27]:
response.choices[0].message.content

'The latest Taylor Swift album is **The Tortured Poets Department**, released on April 19, 2024.'